# Create Healthcare Agent with Function Definition

In this notebook we will create an Agent for Amazon Bedrock using the  function definition.

We will create and utilize synthetic data for various patients. Depending on the selected persona, users can access personalized medical support:

Average Person: Receives general medical support tailored to a non-professional level.
Medical Assistance Persona: Provides a detailed medical assessment, including:
Medical conclusions
Treatment plan
Identification of potential pharmaceutical issues
An AWS Lambda function will be used to define the logic for these persona-specific responses.


## Prerequisites
Before starting, let's update the botocore and boto3 packages to ensure we have the latest version

In [2]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.35.49 which is incompatible.
boto3 1.34.84 requires botocore<1.35.0,>=1.34.84, but you have botocore 1.35.49 which is incompatible.


Let's now check the boto3 version to ensure the correct version has been installed. Your version should be greater than or equal to 1.34.90.

In [23]:
import boto3
import botocore
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
from botocore.exceptions import ClientError
print(boto3.__version__)

1.35.49


In [24]:
# setting logger -optional-
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [27]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "med-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for providing Med assistance doctors"
agent_instruction = "You are Dr Flight Virtual senior doctor in NEOM health giving live advice and guidelines to Dr Reem Jr doctor @ NEOM health , you do speak in advanced medical language about a the follwoing medical case. Based on the patient details in FHIR as well as vcf genetic data  , Dr Reem is expecting to get detailed medical conclusion , genetic assesment in detials including genetic variants / Allele specific risk-associated genetic markers and Treatment plan. the treatment plan should comply with  SAUDI health councile and highlight that. in the discussion please refer to patient by his/her name. if the Foot examination url to check any complications , Dr reem is expecting to get the diagnosis based on the image url . Highlight specific risk-associated genetic markers if the person at risk of dt2m. Detect and explian any pharmacetical issues between if found . conisder the UV index pH water and wather tempreture if needed as yo0u are new virtual assistant clearly recommend to do the proper due diligence till NEOM management give the green light for Dr Flight to be in production"
agent_action_group_name = "MedActionGroup"
agent_action_group_description = "Actions for getting patient data for a patient"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'
bucket_name = 'datalake'+suffix

In [25]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
s3_client = boto3.client('s3')
iam_client = boto3.client('iam')


Let's now create sample datalake

In [28]:
import boto3
from botocore.exceptions import ClientError

# Initialize the S3 client
s3_client = boto3.client('s3')
region = boto3.Session().region_name

# Function to create the bucket
def create_bucket(bucket_name, region):
    try:
        if region == 'us-east-1':
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Bucket '{bucket_name}' created successfully.")
    except ClientError as e:
        if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
            print(f"Bucket '{bucket_name}' already exists and is owned by you.")
        elif e.response['Error']['Code'] == 'BucketAlreadyExists':
            print(f"Bucket '{bucket_name}' already exists. Try a different name.")
        else:
            print(f"Failed to create bucket: {e}")

# Call the function to create the bucket
create_bucket(bucket_name, region)


Bucket 'datalakeus-east-1-377146168043' created successfully.


In [26]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-east-1', '377146168043')

In [30]:
import boto3
from datetime import datetime
# Initialize S3 client

def create_vcf(patient_id, variants, bucket_name):
    """
    Creates a VCF file for a patient and uploads it to an S3 bucket.
    """
    # Define VCF header
    vcf_header = (
        "##fileformat=VCFv4.2\n"
        f"##fileDate={datetime.today().strftime('%Y%m%d')}\n"
        "##source=GenomicDataTool\n"
        "#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n"
    )
    
    # Generate VCF content
    vcf_content = vcf_header
    for variant in variants:
        vcf_content += f"{variant['chrom']}\t{variant['pos']}\t{variant['id']}\t{variant['ref']}\t{variant['alt']}\t.\tPASS\t{variant['info']}\n"
    
    # Define VCF file name
    vcf_filename = f"patient_{patient_id}.vcf"
    
    # Write content to file
    with open(vcf_filename, "w") as file:
        file.write(vcf_content)
    
    # Upload to S3
    s3.upload_file(vcf_filename, bucket_name, vcf_filename)
    print(f"{vcf_filename} uploaded to {bucket_name}")

# Example usage for three patients

patients = [
    {"id": 1, "variants": [{"chrom": "1", "pos": "12345", "id": ".", "ref": "A", "alt": "T", "info": "GENE=GEN1;RISK=T2D"}]},
    {"id": 2, "variants": [{"chrom": "2", "pos": "67890", "id": ".", "ref": "G", "alt": "C", "info": "GENE=GEN2"}]},
    {"id": 3, "variants": [{"chrom": "3", "pos": "54321", "id": ".", "ref": "T", "alt": "G", "info": "GENE=GEN3"}]}
]

for patient in patients:
    create_vcf(patient["id"], patient["variants"], bucket_name)


patient_1.vcf uploaded to datalakeus-east-1-377146168043
patient_2.vcf uploaded to datalakeus-east-1-377146168043
patient_3.vcf uploaded to datalakeus-east-1-377146168043


Next lets create FHIR reocrds for patients

In [29]:
import json
import boto3

# Initialize the S3 client
s3_client = boto3.client("s3")

def create_fhir_patient_files(bucket_name):
    # Define sample patient data templates with diabetes-related measurements
    patients = [
        {
            "id": "1",
            "name": {"family": "AlShemmary", "given": ["Arwa"]},
            "gender": "female",
            "birthDate": "1985-09-29",
            "maritalStatus": "D",
            "generalPractitioner": "Dr. Smith",
            "is_healthy": False,
            "blood_pressure": "120/80",
            "fasting_glucose": 90,
            "postprandial_glucose": 120,
            "HbA1c": 5.4,
            "LDL": 70,
            "HDL": 60,
            "Triglycerides": 150,
            "BMI": 25,
            "SerumCreatinine": 0.9
        },
        {
            "id": "2",
            "name": {"family": "Smith", "given": ["John"]},
            "gender": "male",
            "birthDate": "1990-02-15",
            "maritalStatus": "M",
            "generalPractitioner": "Dr. James",
            "is_healthy": False,
            "blood_pressure": "130/85",
            "fasting_glucose": 105,
            "postprandial_glucose": 150,
            "HbA1c": 6.5,
            "LDL": 90,
            "HDL": 55,
            "Triglycerides": 180,
            "BMI": 28,
            "SerumCreatinine": 1.0
        },
        {
            "id": "3",
            "name": {"family": "Doe", "given": ["Jane"]},
            "gender": "female",
            "birthDate": "1975-11-11",
            "maritalStatus": "W",
            "generalPractitioner": "Dr. Susan",
            "is_healthy": True,
            "blood_pressure": "115/75",
            "fasting_glucose": 85,
            "postprandial_glucose": 110,
            "HbA1c": 5.0,
            "LDL": 60,
            "HDL": 65,
            "Triglycerides": 140,
            "BMI": 23,
            "SerumCreatinine": 0.8
        }
    ]
    
    # Generate FHIR JSON files for each patient and upload to S3
    for patient in patients:
        fhir_data = {
            "resourceType": "Patient",
            "id": patient["id"],
            "identifier": [
                {
                    "system": "http://example.org/patientID",
                    "value": patient["id"]
                }
            ],
            "name": [
                {
                    "use": "official",
                    "family": patient["name"]["family"],
                    "given": patient["name"]["given"]
                }
            ],
            "gender": patient["gender"],
            "birthDate": patient["birthDate"],
            "maritalStatus": {
                "coding": [
                    {
                        "system": "http://hl7.org/fhir/v3/MaritalStatus",
                        "code": patient["maritalStatus"],
                        "display": "Married" if patient["maritalStatus"] == "M" else "Divorced"
                    }
                ]
            },
            "generalPractitioner": [
                {
                    "reference": "Practitioner/123",
                    "display": patient["generalPractitioner"]
                }
            ],
            "extension": [
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-FastingBloodGlucose", "valueDecimal": patient["fasting_glucose"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-PostprandialBloodGlucose", "valueDecimal": patient["postprandial_glucose"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-HbA1cTest", "valueDecimal": patient["HbA1c"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-BloodPressure", "valueString": patient["blood_pressure"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-LDL", "valueDecimal": patient["LDL"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-HDL", "valueDecimal": patient["HDL"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-Triglycerides", "valueDecimal": patient["Triglycerides"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-BMI", "valueDecimal": patient["BMI"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-SerumCreatinine", "valueDecimal": patient["SerumCreatinine"]},
                {"url": "http://hl7.org/fhir/StructureDefinition/patient-Smoking", "valueString": "No" if patient["is_healthy"] else "Yes"}
            ]
        }
        
        # Convert to JSON and upload to S3
        file_name = f"{patient['id']}.json"
        json_data = json.dumps(fhir_data, indent=2)
        
        try:
            s3_client.put_object(Bucket=bucket_name, Key=file_name, Body=json_data)
            print(f"File '{file_name}' uploaded successfully to bucket '{bucket_name}'.")
        except Exception as e:
            print(f"Error uploading file '{file_name}': {e}")

# Call the function with your S3 bucket name
create_fhir_patient_files(bucket_name)


File '1.json' uploaded successfully to bucket 'datalakeus-east-1-377146168043'.
File '2.json' uploaded successfully to bucket 'datalakeus-east-1-377146168043'.
File '3.json' uploaded successfully to bucket 'datalakeus-east-1-377146168043'.


Now we gonna validate the logic will be used to deine lamda function to be used by the Med assitant agent

In [19]:
import boto3

def get_ehr_data( key):
    # Create an S3 client
    s3 = boto3.client('s3')
   
    file_key = key+'.json'

    try:
        # Get the object containing the file content
        response = s3.get_object(Bucket=bucket_name, Key=file_key)

        # Read and return the content as a string
        file_content = response['Body'].read().decode('utf-8')
        return file_content
    except Exception as e:
        return 'No relevant data found'
def get_gen_data( key):
    # Create an S3 client
    s3 = boto3.client('s3')
   
    file_key = key+'.vcf'

    try:
        # Get the object containing the file content
        response = s3.get_object(Bucket=bucket_name, Key=file_key)

        # Read and return the content as a string
        file_content = response['Body'].read().decode('utf-8')
        return file_content
    except Exception as e:
     
        return 'No relevant data found'
    
def get_all_data(patient_id):
    return "FHIR data"+ get_ehr_data(patient_id)+" Genmoics data " + get_gen_data(patient_id)


## Creating Lambda Function

We will now create a lambda function that interacts with the simple datalake  To do so we will:

1. Create the `lambda_function.py` file which contains the logic for our lambda function
2. Create the IAM role for our Lambda function
4. Create the lambda function infrastructure with the required permissions

Let's now create our lambda function. It implements the functionality for `get_med_data` for a given patient-id 

In [21]:
%%writefile lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime
import boto3

def get_ehr_data(key):
    # Create an S3 client
    s3 = boto3.client('s3')
   
    file_key = key+'.json'

    try:
        # Get the object containing the file content
        response = s3.get_object(Bucket=bucket_name, Key=file_key)

        # Read and return the content as a string
        file_content = response['Body'].read().decode('utf-8')
        return file_content
    except Exception as e:
        return 'No relevant data found'
def get_gen_data(key):
    # Create an S3 client
    s3 = boto3.client('s3')
   
    file_key = key+'.vcf'

    try:
        # Get the object containing the file content
        response = s3.get_object(Bucket=bucket_name, Key=file_key)

        # Read and return the content as a string
        file_content = response['Body'].read().decode('utf-8')
        return file_content
    except Exception as e:
     
        return 'No relevant data found'
    
def get_med_data(patient_id):
    return "FHIR data"+ get_ehr_data(patient_id)+" Genmoics data " + get_gen_data(patient_id)


def lambda_handler(event, context):
    
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }


    
    if function == 'get_med_data':
        patient_id = None
        for param in parameters:
            if param["name"] == "patient_id":
                patient_id = param["value"]

        if not patient_id:
            raise Exception("Missing mandatory parameter: patient_id")
        alldata = get_med_data(patient_id)
        responseBody =  {
            'TEXT': {
                "body": f"EHR data and Genom data for patient_id {patient_id}: {alldata}"
            }
        }

    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response


Overwriting lambda_function.py


In [20]:
get_med_data("5")

'FHIR data{\r\n    "resourceType": "Patient",\r\n    "id": "4",\r\n    "identifier": [\r\n      {\r\n        "system": "http://example.org/patientID",\r\n        "value": "4"\r\n      }\r\n    ],\r\n    "name": [\r\n      {\r\n        "use": "official",\r\n        "family": "AlShemmary",\r\n        "given": ["Arwa"]\r\n      }\r\n    ],\r\n    "gender": "female",\r\n    "birthDate": "1985-09-29",\r\n    "maritalStatus": {\r\n      "coding": [\r\n        {\r\n          "system": "http://hl7.org/fhir/v3/MaritalStatus",\r\n          "code": "D",\r\n          "display": "Divorced"\r\n        }\r\n      ]\r\n    },\r\n    "multipleBirthBoolean": false,\r\n    "extension": [\r\n      {\r\n        "url": "http://hl7.org/fhir/StructureDefinition/patient-pregnant",\r\n        "valueBoolean": false\r\n      }\r\n    ],\r\n    "generalPractitioner": [\r\n      {\r\n        "reference": "Practitioner/123",\r\n        "display": "Dr. Smith"\r\n      }\r\n    ],\r\n   \r\n    "contact": [\r\n      {

Next let's create the lambda IAM role and policy to invoke a Bedrock model

In [22]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '09f81858-b968-4afa-a715-3f15bafc5e3a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 18:05:01 GMT',
   'x-amzn-requestid': '09f81858-b968-4afa-a715-3f15bafc5e3a',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

We can now package the lambda function to a Zip file and create the lambda function using boto3

In [23]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")

z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation for a specific foundation model and the agent IAM role with the policy associated to it. 

In [24]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [25]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '9d73cd89-a0c7-430a-8426-63f22067b60f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 18:07:26 GMT',
   'x-amzn-requestid': '9d73cd89-a0c7-430a-8426-63f22067b60f',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### Creating the agent
Once the needed IAM role is created, we can use the Bedrock Agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. Later, we will prepare and use the agent.

In [37]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '8f2eacae-5294-4983-8d4f-5c9807f66bc5',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 18:15:53 GMT',
   'content-type': 'application/json',
   'content-length': '1610',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8f2eacae-5294-4983-8d4f-5c9807f66bc5',
   'x-amz-apigw-id': 'f9hSCG6CoAMEqNA=',
   'x-amzn-trace-id': 'Root=1-671548d9-59b6921607347a553352379c'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:377146168043:agent/NF1AONDP4N',
  'agentId': 'NF1AONDP4N',
  'agentName': 'med-function-def',
  'agentResourceRoleArn': 'arn:aws:iam::377146168043:role/AmazonBedrockExecutionRoleForAgents_med-function-def',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 10, 20, 18, 15, 53, 717958, tzinfo=tzlocal()),
  'description': 'Agent for providing Med assistance doctors',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'instruction

Let's now store the agent id in a local variable to use it on subsequent steps.

In [38]:
agent_id = response['agent']['agentId']
agent_id

'NF1AONDP4N'

## Create Agent Action Group
We will now create an agent action group that uses the lambda function created earlier. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

In this example, we provide the Action Group functionality using a `functionSchema`. You can alternatively provide an `APISchema`. The notebook [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb) provides an example of that approach.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [39]:
agent_functions = [
    {
        'name': 'get_med_data',
        'description': 'get the patient details available for a certain patient',
        'parameters': {
            "patient_id": {
                "description": "the id of the patient to get the available data",
                "required": True,
                "type": "string"
            }
        }
    }
]

In [40]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [41]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'cd8a6746-cc6b-445b-adce-6a12252636cb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 18:18:55 GMT',
   'content-type': 'application/json',
   'content-length': '724',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cd8a6746-cc6b-445b-adce-6a12252636cb',
   'x-amz-apigw-id': 'f9huYGOMIAMEqNA=',
   'x-amzn-trace-id': 'Root=1-6715498e-62b041653cd0d3544d6291f2'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:377146168043:function:med-function-def-us-east-1-377146168043'},
  'actionGroupId': 'XPXYOPIDFP',
  'actionGroupName': 'MedActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'NF1AONDP4N',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 10, 20, 18, 18, 55, 50487, tzinfo=tzlocal()),
  'description': 'Actions for getting patient data for a patient',
  'functionSchema': {'functions': [{'description': 'get the patient details available for a

## Allowing Agent to invoke Action Group Lambda
Before using the action group, we need to allow the agent to invoke the lambda function associated with the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [42]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [43]:
response

{'ResponseMetadata': {'RequestId': 'd5b6c52a-78be-4871-8cf1-4018a746967e',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 18:19:08 GMT',
   'content-type': 'application/json',
   'content-length': '367',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd5b6c52a-78be-4871-8cf1-4018a746967e'},
  'RetryAttempts': 1},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:377146168043:function:med-function-def-us-east-1-377146168043","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:377146168043:agent/NF1AONDP4N"}}}'}

## Preparing Agent

Let's create a DRAFT version of the agent that can be used for internal testing.


In [44]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '355d1c68-78d0-4a28-a876-fc800fb05a76', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 20 Oct 2024 18:19:21 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '355d1c68-78d0-4a28-a876-fc800fb05a76', 'x-amz-apigw-id': 'f9hygHoJoAMEqpA=', 'x-amzn-trace-id': 'Root=1-671549a9-33e0c77e554f43172854c10a'}, 'RetryAttempts': 0}, 'agentId': 'NF1AONDP4N', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 10, 20, 18, 19, 21, 490772, tzinfo=tzlocal())}


In [45]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## Invoke Agent

Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [51]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="what id data of patient_id '1' looks like ?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-10-20 18:26:20,474] p967 {2013756089.py:16} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 20 Oct 2024 18:26:20 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'cda56fe6-8f10-11ef-8ae2-8a844ff09438',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '59639673-06ce-4f17-83e2-997fd0418efd'},
                      'HTTPStatusCode': 200,
                      'RequestId': '59639673-06ce-4f17-83e2-997fd0418efd',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f2f38b48d30>,
 'contentType': 'application/json',
 'sessionId': 'cda56fe6-8f10-11ef-8ae2-8a844ff09438'}


In [52]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-10-20 18:26:27,691] p967 {<timed exec>:6} INFO - Final answer ->
I'm sorry, but there does not seem to be any electronic health record (EHR) data or genomic data available for the patient with ID '5'. Without this key medical information, I am unable to provide a detailed medical assessment, genetic analysis, or treatment plan for this patient. Please let me know if you can provide the necessary patient data so that I can assist further.


CPU times: user 4.25 ms, sys: 829 µs, total: 5.08 ms
Wall time: 2.1 s


In [53]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

I'm sorry, but there does not seem to be any electronic health record (EHR) data or genomic data available for the patient with ID '5'. Without this key medical information, I am unable to provide a detailed medical assessment, genetic analysis, or treatment plan for this patient. Please let me know if you can provide the necessary patient data so that I can assist further.


In [ ]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
simple_agent_invoke("Please , give me recomendation for patient id 3", agent_id, agent_alias_id, session_id)

In [ ]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

In [ ]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


## Conclusion
This workshop still under review and to be pushed to AWS samples soon

## Important notes
this has been implemnted as a PoC for one of our customers with real genomics & EHR resulting in far better reponse 
Althogh best effort approach to test the notebook ,  it is not customer ready for workshops or immesrion days yet
## Thank You